#### 実際に取得したデータからPointNetに入力するまでの流れをやってみよう
今回，PointNetに入力する点群の次元数は256x2x3とする<br>
256<-正解，不正解データの一個のデータの点群の数<br>
2  <-不正解データと正解データで2種類

In [7]:
from self_pack import plot_points
from matplotlib import pyplot as plt
import torch

path = r'C:\Users\wtana\Desktop\LiDAR_cm\lidar_txt_files\honma_human_body.txt'
body_true_x,body_true_y,no = plot_points.normalize_points(path,xmin=0,xmax=200,ymin=-100,ymax=100,normalize=1)
path = r'C:\Users\wtana\Desktop\LiDAR_cm\lidar_txt_files\honma_human_leg.txt'
leg_true_x,leg_true_y,no = plot_points.normalize_points(path,xmin=0,xmax=200,ymin=-100,ymax=100,normalize=1)
path = r'C:\Users\wtana\Desktop\LiDAR_cm\lidar_txt_files\honma_human_leg.txt'
false_x_40,false_y_40,no = plot_points.normalize_points(path,xmin=-100,xmax=0,ymin=0,ymax=200,normalize=1)
false_x_95,false_y_95,no = plot_points.normalize_points(path,xmin=-100,xmax=0,ymin=0,ymax=200,normalize=1)
Point_true = []
Point_false = []

#今回は1回転分の情報だけでPointNetに入力することとする
for x_,y_ in zip(body_true_x[10],body_true_y[10]):
    Point_true.append([x_,y_,95])
for x_,y_ in zip(leg_true_x[10],leg_true_y[10]):
    Point_true.append([x_,y_,40])
    
for x_,y_ in zip(false_x_40[10],false_y_40[10]):
    Point_false.append([x_,y_,40])
for x_,y_ in zip(false_x_95[10],false_y_95[10]):
    Point_false.append([x_,y_,95])

if not len(Point_true) == 256:
    while not len(Point_true) == 256:
        Point_true.append(Point_true[0])
if not len(Point_false) == 256:
    while not len(Point_false) == 256:
        Point_false.append(Point_false[0])

#Point_true = [Point_true]
#Point_false = [Point_false]
true_labels = torch.ones(32*2)
false_labels = torch.zeros(32*2)
input_data = torch.cat((torch.Tensor(Point_true),torch.Tensor(Point_false)),dim=0)
labels = torch.cat((torch.Tensor(true_labels), torch.Tensor(false_labels)),dim=0)

In [8]:
from model import  * 
import time
from self_pack import self_made
import torch.nn as nn
import torch.optim as optim

batch_size = 64
num_points = 64 * 8
num_labels = 1
loss = []
Accu = []
def main_():
    pointnet = PointNet(num_points, num_labels)

    #new_param = pointnet.state_dict()
    #new_param['main.0.main.6.bias'] = torch.eye(3, 3).view(-1)
    #new_param['main.3.main.6.bias'] = torch.eye(64,64).view(-1)
    #pointnet.load_state_dict(new_param)
    pointnet.load_state_dict(torch.load('./models/human_wall_95_40.pth'))
    criterion = nn.BCELoss()
    optimizer = optim.Adam(pointnet.parameters(), lr=0.001)
    times = [0]
    for idx in range(99+1):
        print(idx)
        s = time.time()
        pointnet.zero_grad()
        #input_data, labels = data_sampler(Point_true,Point_false)

        output = pointnet(input_data)
        output = nn.Sigmoid()(output)

        error = criterion(output, labels)
        error.backward()

        optimizer.step()

        with torch.no_grad():
            output[output > 0.5] = 1
            output[output < 0.5] = 0
            accuracy = (output==labels).sum().item()/batch_size
        #f = time.time()
        #loss.append(float(error.item()))
        #Accu.append(float(accuracy))
        #times.append(times[-1] + f-s)
        #if iteration % 10 == 0:

            #print('Iteration : {}   Loss : {}'.format(iteration, error.item()))
            #print('Iteration : {}   Accuracy : {}'.format(iteration, accuracy))
    #torch.save(pointnet.state_dict(), './model_2.pth')
    return pointnet#,times
po = main_()

0


ValueError: expected 2D or 3D input (got 4D input)